In [9]:
# !pip install selenium
from bs4 import BeautifulSoup
from requests import get
from selenium import webdriver
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
import re
import json
import numpy as np
import time
import random
import os

# Functions

In [2]:
def get_soup(url) :
    driver.get(url)
    content = driver.page_source
    soup = BeautifulSoup(content)
    return soup

In [3]:
def get_links(soup) :
    resultsLinks = soup.find_all('a', class_='card__title-link')
    links = []
    for i in resultsLinks :
        links.append(i.attrs['href'])
    return links

In [4]:
def get_info(soup, link) :
    """Get all info about a property"""
    Link = link
    #dataLayer infos
    dataLayer = soup.find_all('script')[4]
    dataLayer2 = str(dataLayer)[33:-12].strip()
#    print(dataLayer2[0:10])
    if dataLayer2[0] == '{' :
        dataLayer_dict = json.loads(dataLayer2) #[0]
        Id = dataLayer_dict['classified']['id']
        Type = dataLayer_dict['classified']['type']
        Subtype = dataLayer_dict['classified']['subtype']
        Price = dataLayer_dict['classified']['price']
        Zip = dataLayer_dict['classified']['zip']
        ConstructionYear = dataLayer_dict['classified']['building']['constructionYear']
        Condition = dataLayer_dict['classified']['building']['condition']
        Bedrooms = dataLayer_dict['classified']['bedroom']['count']
        Land = dataLayer_dict['classified']['land']['surface']
        Attic = dataLayer_dict['classified']['atticExists']
        Garden = dataLayer_dict['classified']['outdoor']['garden']['surface']
        Terrace = dataLayer_dict['classified']['outdoor']['terrace']['exists']
        ParkingIndoor = dataLayer_dict['classified']['parking']['parkingSpaceCount']['indoor']
        # overview__text
        if len(soup.find_all('div', class_='overview__column')) > 0 :
            bathPath = soup.find_all('div', class_='overview__column')[0].find_all('div', class_='overview__item')
            if len(bathPath) > 1 :
                Bathrooms = bathPath[1].text.replace('\n','').replace(' ','').replace('bathroom','').replace('s','')
            else :
                Bathrooms = np.nan
        else :
            Bathrooms = np.nan
        if len(soup.find_all('div', class_='overview__column')) > 1 :
            areaPath = soup.find_all('div', class_='overview__column')[1].find_all('div', class_='overview__item')
            if len(areaPath) > 0 :
                Area = areaPath[0].text.replace('\n','').replace(' ','').replace('m²squaremeterslivablespace','')
                if 'land' in Area :
                    Area = np.nan
            else:
                Area = np.nan
        else :
            Area = np.nan
        Info = [Id, Type, Subtype, Price, Area, Zip, ConstructionYear, Condition, 
                Bedrooms,Bathrooms, Land, Garden, Terrace, ParkingIndoor, Link]
    else:
        Info = [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, 
                np.nan,np.nan, np.nan, np.nan, np.nan, np.nan] 
        
    return Info

In [5]:
cols=['Id', 'Type', 'Subtype', 'Price', 'Area', 'Zip', 'ConstructionYear', 'Condition', 
            'Bedrooms','Bathrooms', 'Land', 'Garden', 'Terrace', 'ParkingIndoor', 'Link']

# Import previous

In [37]:
df_old = pd.read_csv('data/data.csv')
display(df_old.head())
df_old.info()

,Id,Type,Subtype,Price,Area,Zip,ConstructionYear,Condition,Bedrooms,Bathrooms,Land,Garden,Terrace,ParkingIndoor,Link,Date,Price/m²
0,9515485,apartment,duplex,449000.0,146.0,1140,NaN,as new,4.0,2.0,NaN,NaN,True,2.0,https://www.immoweb.be/en/classified/duplex/fo...,2021-09-14,3075.342466
1,9515448,apartment,apartment,275000.0,93.0,1140,NaN,as new,2.0,1.0,NaN,NaN,True,2.0,https://www.immoweb.be/en/classified/apartment...,2021-09-14,2956.989247
2,9515447,apartment,apartment,190000.0,74.0,1210,1958.0,NaN,2.0,1.0,NaN,NaN,True,NaN,https://www.immoweb.be/en/classified/apartment...,2021-09-14,2567.567568
3,9515435,apartment,apartment,215000.0,70.0,1140,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,https://www.immoweb.be/en/classified/apartment...,2021-09-14,3071.428571
4,9515409,apartment,apartment,400000.0,115.0,1200,NaN,NaN,2.0,1.0,NaN,NaN,NaN,1.0,https://www.immoweb.be/en/classified/apartment...,2021-09-14,3478.260870


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3676 entries, 0 to 3675
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Id                3676 non-null   int64  
 1   Type              3676 non-null   object 
 2   Subtype           3676 non-null   object 
 3   Price             3676 non-null   float64
 4   Area              3676 non-null   float64
 5   Zip               3676 non-null   int64  
 6   ConstructionYear  2599 non-null   float64
 7   Condition         3038 non-null   object 
 8   Bedrooms          3676 non-null   float64
 9   Bathrooms         3676 non-null   float64
 10  Land              940 non-null    float64
 11  Garden            674 non-null    float64
 12  Terrace           2655 non-null   object 
 13  ParkingIndoor     1234 non-null   float64
 14  Link              3676 non-null   object 
 15  Date              3676 non-null   object 
 16  Price/m²          3676 non-null   float64


# Get new 

In [15]:
driver = webdriver.Chrome(executable_path='C:/Users/harri/Documents/Google/Chrome/chromedriver.exe')
if os.path.isfile('data/df_temp.csv'):
    df_temp = pd.read_csv('data/df_temp.csv')
else:
    df_temp = pd.DataFrame(columns=cols)

numberResult = 1
page = 0
Info = []
Id_Active = []
while numberResult > 0 :
    page += 1
    print(page, end='\r')
    urlList= 'https://www.immoweb.be/en/search/house-and-apartment/for-sale/brussels/province?countries=BE&page='\
                + str(page) + '&orderBy=newest'
    # get a page containing a list of results 
    resultList = get_soup(urlList)
    # get the links to each result 
    resultLinks = get_links(resultList)
    numberResult = len(resultLinks) 
    # get the info from the property
    for link in resultLinks:
        ID = int(link.split('/')[9].split('?')[0])
        Id_Active.append(ID)
        if ID not in list(df_old['Id']) and ID not in list(df_temp['Id']):
            resultProp = get_soup(link)
            Info.append(get_info(resultProp, link))
            df_new = pd.DataFrame(Info, columns=cols)
            df_temp = df_temp.append(df_new)
            df_temp = df_temp.drop_duplicates()
            df_temp.to_csv('data/df_temp.csv', index=False)
            time.sleep(random.randint(1,2))
            

In [24]:
# df_new = df_new.dropna()
# # print(cols)
# display(df_new.head(20))
# df_new.to_csv('data/newnew.csv')

,Id,Type,Subtype,Price,Area,Zip,ConstructionYear,Condition,Bedrooms,Bathrooms,Land,Garden,Terrace,ParkingIndoor,Link
0,9515485,apartment,duplex,449000,146,1140,,as new,4,2,,,true,2,https://www.immoweb.be/en/classified/duplex/fo...
1,9515481,apartment,apartment,210000,Floor:2,1030,,,1,1,,,true,,https://www.immoweb.be/en/classified/apartment...
3,9515448,apartment,apartment,275000,93,1140,,as new,2,1,,,true,2,https://www.immoweb.be/en/classified/apartment...
4,9515447,apartment,apartment,190000,74,1210,1958,,2,1,,,true,,https://www.immoweb.be/en/classified/apartment...
6,9515435,apartment,apartment,215000,70,1140,,,2,1,,,,,https://www.immoweb.be/en/classified/apartment...
7,9515409,apartment,apartment,400000,115,1200,,,2,1,,,,1,https://www.immoweb.be/en/classified/apartment...
8,9515396,apartment,apartment,149000,60,1180,1975,as new,1,1,,,true,,https://www.immoweb.be/en/classified/apartment...
9,9515393,apartment,apartment,495000,150,1180,1964,to renovate,3,2,,,true,1,https://www.immoweb.be/en/classified/apartment...
10,9515392,house,apartment block,1850000,420,1040,1910,good,5,4,0,,true,1,https://www.immoweb.be/en/classified/apartment...
11,9515389,house,apartment block,650000,205,1030,1927,good,3,3,79,,true,,https://www.immoweb.be/en/classified/apartment...


# Format new

In [39]:
df_new['Zip'] = df_new['Zip'].apply(int)
df_new['Id'] = df_new['Id'].apply(int)

#df_codes = pd.read_csv('data/codes-communes.csv', sep=';')
today = time.strftime('%Y-%m-%d')
# df_new = df_new.merge(df_codes, how='left', left_on='Zip', right_on='Code postal')

def bx_ok(x) :
    if '19' in x :
        out = 'Bruxelles'
    else :
        out = x
    return out

# df_new['Province'] = df_new['Province'].apply(bx_ok)

df_new = df_new[['Id', 'Type', 'Subtype', 'Price', 'Area', 'Zip', 'ConstructionYear',
       'Condition', 'Bedrooms', 'Bathrooms', 'Land', 'Garden', 'Terrace',
       'ParkingIndoor', 'Link']]

df_new['Area'] =df_new['Area'].replace(r"[^0-9.]", np.nan, regex=True).astype(float)
df_new['Price'] =df_new['Price'].replace(r"[^0-9.]", np.nan, regex=True).astype(float)

df_new = df_new.dropna(subset=['Price',  'Area'])

df_new['Bedrooms'] = df_new['Bedrooms'].replace(r"[^0-9.]", np.nan, regex=True).apply(float)
df_new['Bathrooms'] = df_new['Bathrooms'].replace(r"[^0-9.]", np.nan, regex=True).apply(float)
df_new['Zip'] = df_new['Zip'].apply(str)
df_new['Terrace'] = df_new['Terrace'].replace(np.nan, 'False')

def commune(x) : 
    if x == 'WOLUWE-SAINT-LAMBERT' :
        out = x
    else :
        out = x + ' (Bruxelles)'
    return out

# df_new['Commune'] = df_new['Commune principale'].apply(commune)

df_new['Date'] = today

df_new['Price/m²'] = df_new['Price'] / df_new['Area']

df_new.info()

display(df_new.head())

df_new.to_csv('data/data.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3676 entries, 0 to 5415
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Id                3676 non-null   int64  
 1   Type              3676 non-null   object 
 2   Subtype           3676 non-null   object 
 3   Price             3676 non-null   float64
 4   Area              3676 non-null   float64
 5   Zip               3676 non-null   object 
 6   ConstructionYear  3676 non-null   object 
 7   Condition         3676 non-null   object 
 8   Bedrooms          3676 non-null   float64
 9   Bathrooms         3676 non-null   float64
 10  Land              3676 non-null   object 
 11  Garden            3676 non-null   object 
 12  Terrace           3676 non-null   object 
 13  ParkingIndoor     3676 non-null   object 
 14  Link              3676 non-null   object 
 15  Date              3676 non-null   object 
 16  Price/m²          3676 non-null   float64


<ipython-input-39-1fbb93c85ed0>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['Area'] =df_new['Area'].replace(r"[^0-9.]", np.nan, regex=True).astype(float)
<ipython-input-39-1fbb93c85ed0>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['Price'] =df_new['Price'].replace(r"[^0-9.]", np.nan, regex=True).astype(float)


,Id,Type,Subtype,Price,Area,Zip,ConstructionYear,Condition,Bedrooms,Bathrooms,Land,Garden,Terrace,ParkingIndoor,Link,Date,Price/m²
0,9515485,apartment,duplex,449000.0,146.0,1140,,as new,4.0,2.0,,,true,2,https://www.immoweb.be/en/classified/duplex/fo...,2021-09-14,3075.342466
3,9515448,apartment,apartment,275000.0,93.0,1140,,as new,2.0,1.0,,,true,2,https://www.immoweb.be/en/classified/apartment...,2021-09-14,2956.989247
4,9515447,apartment,apartment,190000.0,74.0,1210,1958,,2.0,1.0,,,true,,https://www.immoweb.be/en/classified/apartment...,2021-09-14,2567.567568
6,9515435,apartment,apartment,215000.0,70.0,1140,,,2.0,1.0,,,,,https://www.immoweb.be/en/classified/apartment...,2021-09-14,3071.428571
7,9515409,apartment,apartment,400000.0,115.0,1200,,,2.0,1.0,,,,1,https://www.immoweb.be/en/classified/apartment...,2021-09-14,3478.260870


# Concat, Remove Sold, & Save

In [40]:
df = pd.concat([df_new, df_old]) #, sort=False)
df_active = pd.DataFrame(Id_Active, columns=['Id'])
df = df_active.merge(df, how='right', on='Id')

df.head()
def garden(x) :
    if x == '' :
        out = False
    else :
        out = True
    return out
df['Garden'] = df['Garden'].apply(garden)
df['Terrace'] = df['Terrace'].apply(garden)
df['Price/m²'] = df['Price'] / df['Area']

df = df.drop_duplicates('Id')

print(df.shape)

df.head()

(3666, 17)


,Id,Type,Subtype,Price,Area,Zip,ConstructionYear,Condition,Bedrooms,Bathrooms,Land,Garden,Terrace,ParkingIndoor,Link,Date,Price/m²
0,9515485,apartment,duplex,449000.0,146.0,1140,,as new,4.0,2.0,,False,True,2,https://www.immoweb.be/en/classified/duplex/fo...,2021-09-14,3075.342466
1,9515448,apartment,apartment,275000.0,93.0,1140,,as new,2.0,1.0,,False,True,2,https://www.immoweb.be/en/classified/apartment...,2021-09-14,2956.989247
2,9515447,apartment,apartment,190000.0,74.0,1210,1958,,2.0,1.0,,False,True,,https://www.immoweb.be/en/classified/apartment...,2021-09-14,2567.567568
3,9515435,apartment,apartment,215000.0,70.0,1140,,,2.0,1.0,,False,False,,https://www.immoweb.be/en/classified/apartment...,2021-09-14,3071.428571
4,9515409,apartment,apartment,400000.0,115.0,1200,,,2.0,1.0,,False,False,1,https://www.immoweb.be/en/classified/apartment...,2021-09-14,3478.260870


In [41]:
df.to_csv('data/data.csv', index=False)
df = df.drop_duplicates(subset='Id')
df.to_csv('data/Archive/data_'+today+'.csv', index=False)
df.info()
# df.tail()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3666 entries, 0 to 3696
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Id                3666 non-null   int64  
 1   Type              3666 non-null   object 
 2   Subtype           3666 non-null   object 
 3   Price             3666 non-null   float64
 4   Area              3666 non-null   float64
 5   Zip               3666 non-null   object 
 6   ConstructionYear  3666 non-null   object 
 7   Condition         3666 non-null   object 
 8   Bedrooms          3666 non-null   float64
 9   Bathrooms         3666 non-null   float64
 10  Land              3666 non-null   object 
 11  Garden            3666 non-null   bool   
 12  Terrace           3666 non-null   bool   
 13  ParkingIndoor     3666 non-null   object 
 14  Link              3666 non-null   object 
 15  Date              3666 non-null   object 
 16  Price/m²          3666 non-null   float64
